In [1]:
# SECDA Benchmark Suite
# This notebook replicates the functionality of the secda_benchmark_suite.sh script

import os
import subprocess
import json
from datetime import datetime
import signal
import ipywidgets as widgets
from IPython.display import display

# Load conda_path from config.json
with open('../../config.json') as config_file:
    config = json.load(config_file)

conda_path = config['conda_path']

# Run configure_benchmark.py to generate board_config.json
subprocess.run("python3 scripts/configure_benchmark.py -c 1", shell=True)

# Load board configurations from board_config.json
with open('./generated/board_config.json') as board_config_file:
    board_config = json.load(board_config_file)

board = board_config['board']
board_user = board_config['board_user']
board_hostname = board_config['board_hostname']
board_dir = board_config['board_dir']
board_port = board_config['board_port']
bench_dir = os.path.join(board_dir, 'benchmark_suite')

# Define helper functions
def help_function():
    print("""
    Usage: Run the cells in the notebook sequentially to replicate the functionality of the secda_benchmark_suite.sh script.
    """)

def create_dir():
    subprocess.run(f"ssh -o LogLevel=QUIET -t -p {board_port} {board_user}@{board_hostname} 'mkdir -p {bench_dir} && mkdir -p {board_dir}/bitstreams && mkdir -p {bench_dir}/bins && mkdir -p {bench_dir}/models'", shell=True)
    subprocess.run(f"rsync -r -avz -e 'ssh -p {board_port}' ./model_gen/models {board_user}@{board_hostname}:{bench_dir}/", shell=True)
    subprocess.run(f"rsync -r -avz -e 'ssh -p {board_port}' ./bitstreams/{board}/ {board_user}@{board_hostname}:{board_dir}/bitstreams/", shell=True)
    subprocess.run(f"rsync -q -r -avz -e 'ssh -p {board_port}' ./scripts/fpga_scripts/ {board_user}@{board_hostname}:{board_dir}/scripts/", shell=True)
    print("Initialization Done")

def ctrl_c_handler(signum, frame):
    print("Exiting")
    exit(1)

signal.signal(signal.SIGINT, ctrl_c_handler)

# Define variables
skip_bench = 0
bin_gen = 0
skip_inf_diff = 0
collect_power = 0
test_run = 0
init = 0
name = ""
now = datetime.now().strftime("%Y_%m_%d_%H_%M")

# Configuration widgets
skip_bench_widget = widgets.Checkbox(value=False, description='Skip running experiment')
bin_gen_widget = widgets.Checkbox(value=False, description='Generate binaries')
skip_inf_diff_widget = widgets.Checkbox(value=False, description='Skip inference difference checks')
collect_power_widget = widgets.Checkbox(value=False, description='Power collection')
test_run_widget = widgets.Checkbox(value=False, description='Test run')
init_widget = widgets.Checkbox(value=False, description='Initialize the board')
name_widget = widgets.Text(value='', description='Name of the experiment')

display(skip_bench_widget, bin_gen_widget, skip_inf_diff_widget, collect_power_widget, test_run_widget, init_widget, name_widget)


Loading Board Config


Checkbox(value=False, description='Skip running experiment')

Checkbox(value=False, description='Generate binaries')

Checkbox(value=False, description='Skip inference difference checks')

Checkbox(value=False, description='Power collection')

Checkbox(value=False, description='Test run')

Checkbox(value=False, description='Initialize the board')

Text(value='', description='Name of the experiment')

In [ ]:
# Set the name of the experiment
if name_widget.value == "":
    name = f"run_{now}"
else:
    name = f"{name_widget.value}_{now}"

# Display configurations
print("-----------------------------------------------------------")
print("-- SECDA-TFLite Benchmark Suite --")
print("-----------------------------------------------------------")
print("Configurations")
print("--------------")
print(f"Board: {board}")
print(f"Board User: {board_user}")
print(f"Board Hostname: {board_hostname}")
print(f"Board Benchmark Dir: {bench_dir}")
print(f"Skip Bench: {skip_bench_widget.value}")
print(f"Bin Gen: {bin_gen_widget.value}")
print(f"Skip Inf Diff: {skip_inf_diff_widget.value}")
print(f"Collect Power: {collect_power_widget.value}")
print(f"Test Run: {test_run_widget.value}")
print(f"Name: {name}")
print("-----------------------------------------------------------")

# Initialize SECDA-TFLite Benchmark Suite
print("-----------------------------------------------------------")
print("Initializing SECDA-TFLite Benchmark Suite")
print("-----------------------------------------------------------")
if not skip_bench_widget.value:
    print("Clearing cache")
    subprocess.run("rm -rf ./tmp", shell=True)

if init_widget.value:
    create_dir()
print("-----------------------------------------------------------")

# Generate binaries and experiment configurations
print("-----------------------------------------------------------")
print("Configuring Benchmark")
print("-----------------------------------------------------------")
subprocess.run(f"python3 scripts/configure_benchmark.py -e 1 -b {int(bin_gen_widget.value)}", shell=True)

if bin_gen_widget.value:
    print("-----------------------------------------------------------")
    print("Generating Binaries")
    subprocess.run("pwd && source ./generated/gen_bins.sh", shell=True)
    print("-----------------------------------------------------------")

# subprocess.run("source ./generated/configs.sh", shell=True)
# length = len(os.environ.get('hw_array', '').split())

# subprocess.run(f"source {conda_path}/activate tf", shell=True)

# if not skip_bench_widget.value:
#     print("-----------------------------------------------------------")
#     print("Transferring Experiment Configurations to Target Device")
#     subprocess.run(f"rsync -q -r -avz -e 'ssh -p {board_port}' ./generated/configs.sh {board_user}@{board_hostname}:{bench_dir}/", shell=True)
#     subprocess.run(f"rsync -q -r -avz -e 'ssh -p {board_port}' ./generated/run_collect.sh {board_user}@{board_hostname}:{bench_dir}/", shell=True)
#     subprocess.run(f"ssh -o LogLevel=QUIET -t -p {board_port} {board_user}@{board_hostname} 'cd {bench_dir}/ && chmod +x ./*.sh'", shell=True)

#     if collect_power_widget.value:
#         print("-----------------------------------------------------------")
#         print("Initializing Power Capture")
#         subprocess.Popen(f"python3 scripts/record_power.py {name}", shell=True)
#         with open('/tmp/record_power.py.pid', 'w') as pid_file:
#             pid_file.write(str(os.getpid()))
#         print("-----------------------------------------------------------")

#     print("-----------------------------------------------------------")
#     print("Running Experiments")
#     print("-----------------------------------------------------------")
#     subprocess.run(f"ssh -o LogLevel=QUIET -t -p {board_port} {board_user}@{board_hostname} 'cd {bench_dir}/ && ./run_collect.sh {int(process_on_fpga)} {int(skip_inf_diff_widget.value)} {int(collect_power_widget.value)} {int(test_run_widget.value)}'", shell=True)

#     if not test_run_widget.value:
#         print("Transferring Results to Host")
#         subprocess.run(f"rsync --mkpath -q -r -av -e 'ssh -p {board_port}' {board_user}@{board_hostname}:{bench_dir}/tmp ./", shell=True)
#         subprocess.run(f"rsync --mkpath -q -r -av -e 'ssh -p {board_port}' {board_user}@{board_hostname}:{bench_dir}/tmp ./tmp/{name}/", shell=True)
#     print("-----------------------------------------------------------")

#     if collect_power_widget.value:
#         if os.path.exists('/tmp/record_power.py.pid'):
#             with open('/tmp/record_power.py.pid') as pid_file:
#                 os.kill(int(pid_file.read()), signal.SIGTERM)
#             print("-----------------------------------------------------------")
#             print("Power Capture Done")
#             print("-----------------------------------------------------------")
#             print("Processing Power Data")
#             print("-----------------------------------------------------------")
#             subprocess.run(f"python3 scripts/process_power.py {name} {length}", shell=True)
#             print("Power Processing Done")
#             print("-----------------------------------------------------------")
#         else:
#             print("/tmp/record_power.py.pid not found")
#         print(f"Simple csv: ./files/{name}_clean.csv")
#         print("-----------------------------------------------------------")

# if not test_run_widget.value:
#     print("-----------------------------------------------------------")
#     print("Post Processing")
#     prev_failed = 0
#     prev_hw = ""
#     length = len(os.environ.get('hw_array', '').split())
#     for i in range(length):
#         index = i + 1
#         HW = os.environ.get(f'hw_array[{i}]')
#         MODEL = os.environ.get(f'model_array[{i}]')
#         THREAD = os.environ.get(f'thread_array[{i}]')
#         NUM_RUN = os.environ.get(f'num_run_array[{i}]')
#         VERSION = os.environ.get(f'version_array[{i}]')
#         DEL_VERSION = os.environ.get(f'del_version_array[{i}]')
#         DEL = os.environ.get(f'del_array[{i}]')
#         prev_failed = 0
#         prev_hw = HW
#         runname = f"{HW}_{VERSION}_{DEL}_{DEL_VERSION}_{MODEL}_{THREAD}_{NUM_RUN}"
#         if i % 1 == 0:
#             print("========================================================================")
#             print(f"{runname} {index}/{length}")
#             print("========================================================================")

#         valid = 1
#         if HW != "CPU" and not skip_inf_diff_widget.value:
#             if not os.path.exists(f'tmp/{runname}_id.txt'):
#                 valid = 0
#                 print(f"Correct Check File Missing for: {runname}")
#             else:
#                 subprocess.run(f"python3 scripts/fpga_scripts/check_valid.py tmp/{runname}_id.txt", shell=True)
#                 if subprocess.returncode != 0:
#                     valid = 0
#                     print(f"Correctness Check Failed {runname}")

#         subprocess.run(f"python3 scripts/process_run.py {MODEL} {THREAD} {NUM_RUN} {HW} {VERSION} {DEL} {DEL_VERSION} {valid} {name}", shell=True)
#         if subprocess.returncode != 0:
#             prev_failed = 1
#             print("Process Run Failed")
#             continue

#     subprocess.run(f"python3 scripts/process_all_runs.py {name}", shell=True)
#     print("-----------------------------------------------------------")
#     print("Post Processing Done")
#     print("-----------------------------------------------------------")

# print("-----------------------------------------------------------")
# print("Exiting SECDA-TFLite Benchmark Suite")
# print("-----------------------------------------------------------")

-----------------------------------------------------------
-- SECDA-TFLite Benchmark Suite --
-----------------------------------------------------------
Configurations
--------------
Board: Z1
Board User: xilinx
Board Hostname: jharis.ddns.net
Board Benchmark Dir: /home/xilinx/Workspace/secda_tflite/benchmark_suite
Skip Bench: False
Bin Gen: True
Skip Inf Diff: False
Collect Power: False
Test Run: True
Name: run_2025_02_17_00_59
-----------------------------------------------------------
-----------------------------------------------------------
Initializing SECDA-TFLite Benchmark Suite
-----------------------------------------------------------
Clearing cache
-----------------------------------------------------------
-----------------------------------------------------------
Configuring Benchmark
-----------------------------------------------------------
Creating Experiment
Generating benchmark
Generating Binaries
-----------------------------------------------------------
Gener

FileNotFoundError: [Errno 2] No such file or directory: 'pwd && source ./generated/gen_bins.sh'